<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220105_sm_know_rf_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/train/*.csv'))]
know_test = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/test/*.csv'))]
train_know = pd.read_csv('/content/gdrive/MyDrive/know/df_1')
test_know = pd.read_csv('/content/gdrive/MyDrive/know/df_test')

In [4]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [5]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [6]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []
year_encoder = {}

for year, train, test in zip(years, know_train, know_test):
    print(year)
    encoders = {}
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
        except:
            encoder = LabelEncoder()
            train[col] = train[col].map(str)
            train[col] = encoder.fit_transform(train[col])
            encoders[col] = encoder  
    year_encoder[year] = encoders

    train = pd.merge(train, train_know[['idx', 'label_answer']], how='left', left_on='idx', right_on='idx')
    train = train[train['label_answer'].notnull()]

    feature = train.columns.drop(['idx', 'knowcode'])

    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in test.columns:
      try:
          test[col] = test[col].map(int)
      except:
          encoder = year_encoder[year][col]
          test[col] = test[col].map(str)
          category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
          test[col] = test[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
    test = pd.merge(test, test_know[['idx', 'label_answer']], how='left', left_on='idx', right_on='idx')
    feature = test.columns.drop(['idx'])
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [7]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기

In [8]:
submission['knowcode'] = rf_predicts

In [9]:
submission

,idx,knowcode
0,0,29401
1,1,9999999
2,2,9999999
3,3,121102
4,4,412003
...,...,...
35226,35244,811101
35227,35245,32301
35228,35246,833001
35229,35247,154101


In [10]:
submission.to_csv('submission.csv', index=False)

In [11]:
train_know

,idx,text,knowcode,target_label,label_answer
0,19512,없다 현대사회에서 지적재산권등에 관한 대중의 관심이 늘기때문에 없다 없다 없다 전자공학,221401,81,81
1,29499,환자 수 증가로 인한 업무량 증가 전산 시스템 발달로 인해 사람이 필요 없어질 수도...,307401,493,493
2,14916,생활수준이 향상되어서 없다 캔버스 이젤 붓 화통 없다 없다 미술학,414101,470,470
3,16342,건설 업체가 너무 많아서 없다 공구 없다 없다 없다,701301,66,453
4,19430,없다 없다 사무직 없다 없다 문,420901,126,265
...,...,...,...,...,...
17612,13651,현상유지 없다 통신장비 없다 없다 기계,136001,415,415
17613,5230,"하역 및 적재관련 업무가 단순하고 쉽기 때문에 꾸준히 늘어날 것 같음 없다 손, 포...",624401,533,533
17614,31437,없다 앞으로 경기가 풀리면 자동차 수요가 늘어나게 되므로 관광,817201,172,237
17615,18842,없다 온라인 판매가 급증하고 있어서 없다 없다 없다 상,617101,444,432
